# FIFA Player Rating Predictions
# Name: Nana Kwaku Afriyie Ampadu-Boateng
# ID: 42202026
# References and Resources: https://youtube.com/playlist?list=PL-u09-6gP5ZPOfSPTto4BIDwky-8aP4rQ&si=sXccxUsEi3rnEBLC
# Help: I received some assistance from Alvin Broke in class of 2025

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import pickle
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from scipy.stats import randint, uniform
from sklearn.model_selection import train_test_split

In [2]:
males_legacy_df = pd.read_csv('male_players (legacy) (1).csv',low_memory = False)

In [3]:
males_legacy_df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [4]:
def remove_weak_correlation(dataframe, response_column, min_correlation=0.5):
    numeric_columns = dataframe.select_dtypes(include=['number'])
    low_corr_columns = []

    for col in numeric_columns.columns:
        if col != response_column:
            correlation = numeric_columns[col].corr(numeric_columns[response_column])
            if abs(correlation) < min_correlation or pd.isna(correlation):
                low_corr_columns.append(col)

    cleaned_dataframe = dataframe.drop(columns=low_corr_columns)
    return cleaned_dataframe

cleaned_data = remove_weak_correlation(males_legacy_df, 'overall')


/opt/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [5]:
cleaned_data.head()

,player_url,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,1987-06-24,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,1985-02-05,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,1984-01-23,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,1981-10-03,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,1986-03-27,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [6]:
def encode_preferred_foot(dataframe):
    encoded_df = dataframe.copy()
    encoded_df = pd.get_dummies(encoded_df, columns=['preferred_foot'], prefix='foot')
    foot_cols = [column for column in encoded_df.columns if column.startswith('foot_')]
    for column in foot_cols:
        encoded_df[column] = encoded_df[column].astype(int)
    return encoded_df


encoded_data = encode_preferred_foot(cleaned_data)


In [7]:
encoded_data.head()

,player_url,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,foot_Left,foot_Right
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,1987-06-24,...,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png,1,0
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,1985-02-05,...,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png,0,1
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,1984-01-23,...,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png,1,0
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,1981-10-03,...,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png,0,1
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,1986-03-27,...,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png,0,1


In [8]:
def drop_columns_with_high_nulls(data_to_clean, null_threshold=0.3):
    # Calculate the number of non-null values required to retain the column
    threshold_count = int((1 - null_threshold) * len(data_to_clean))
    
    # Drop columns exceeding the specified null threshold
    cleaned_data = data_to_clean.dropna(axis=1, thresh=threshold_count)
    
    return cleaned_data




In [9]:
result_cleaned = drop_columns_with_high_nulls(encoded_data, null_threshold=0.3)

# Print the cleaned DataFrame
result_cleaned.head()

,player_url,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,foot_Left,foot_Right
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,1987-06-24,...,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png,1,0
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,1985-02-05,...,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png,0,1
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,1984-01-23,...,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png,1,0
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,1981-10-03,...,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png,0,1
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,1986-03-27,...,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png,0,1


In [10]:
def remove_categorical_features(dataset):
    cat_cols = dataset.select_dtypes(include=['object']).columns
    updated_dataset = dataset.drop(columns=cat_cols)
    return updated_dataset

encoded_data = remove_categorical_features(encoded_data)




In [11]:

encoded_data.head()

,overall,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,physic,movement_reactions,mentality_composure,foot_Left,foot_Right
0,93,95,100500000.0,550000.0,NaN,86.0,96.0,63.0,94,NaN,1,0
1,92,92,79000000.0,375000.0,NaN,81.0,91.0,79.0,90,NaN,0,1
2,90,90,54500000.0,275000.0,NaN,83.0,92.0,64.0,89,NaN,1,0
3,90,90,52500000.0,275000.0,NaN,81.0,86.0,86.0,85,NaN,0,1
4,90,90,63500000.0,300000.0,NaN,NaN,NaN,NaN,89,NaN,0,1


# Imputing the dataset

In [12]:
#Inserting the median value where there are Null values
def impute_with_median(df):
    for column in df.columns:
        median_value = df[column].median(skipna=True)
        df[column].fillna(median_value, inplace=True)
    return df
imputed_data = impute_with_median(encoded_data)

In [13]:
imputed_data = imputed_data.drop(columns=['release_clause_eur'])

In [14]:
imputed_data.info()
imputed_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   overall              161583 non-null  int64  
 1   potential            161583 non-null  int64  
 2   value_eur            161583 non-null  float64
 3   wage_eur             161583 non-null  float64
 4   passing              161583 non-null  float64
 5   dribbling            161583 non-null  float64
 6   physic               161583 non-null  float64
 7   movement_reactions   161583 non-null  int64  
 8   mentality_composure  161583 non-null  float64
 9   foot_Left            161583 non-null  int64  
 10  foot_Right           161583 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 13.6 MB


,overall,potential,value_eur,wage_eur,passing,dribbling,physic,movement_reactions,mentality_composure,foot_Left,foot_Right
0,93,95,100500000.0,550000.0,86.0,96.0,63.0,94,59.0,1,0
1,92,92,79000000.0,375000.0,81.0,91.0,79.0,90,59.0,0,1
2,90,90,54500000.0,275000.0,83.0,92.0,64.0,89,59.0,1,0
3,90,90,52500000.0,275000.0,81.0,86.0,86.0,85,59.0,0,1
4,90,90,63500000.0,300000.0,58.0,63.0,66.0,89,59.0,0,1


# Feature Engineering

In [15]:
#Scaling with Standard Scaler
def standardize_features(dataset, target):
    data_copy = dataset.copy()
    features_to_standardize = [feature for feature in data_copy.columns if feature != target]
    standardizer = StandardScaler()
    data_copy[features_to_standardize] = standardizer.fit_transform(data_copy[features_to_standardize])

    return data_copy, standardizer


standardized_data, fitted_scaler = standardize_features(imputed_data, 'overall')
final_result = standardized_data


In [16]:
final_result.head()

,overall,potential,value_eur,wage_eur,passing,dribbling,physic,movement_reactions,mentality_composure,foot_Left,foot_Right
0,93,3.875315,16.447428,24.697104,2.926570,3.480487,-0.211839,3.535775,0.085565,1.807362,-1.807362
1,92,3.396013,12.846177,16.681825,2.421008,2.965866,1.529711,3.099571,0.085565,-0.553293,0.553293
2,90,3.076478,8.742427,12.101666,2.623233,3.068791,-0.102992,2.990520,0.085565,1.807362,-1.807362
3,90,3.076478,8.407427,12.101666,2.421008,2.451245,2.291639,2.554317,0.085565,-0.553293,0.553293
4,90,3.076478,10.249927,13.246706,0.095424,0.083989,0.114701,2.990520,0.085565,-0.553293,0.553293


# Training The Model

In [17]:
# Separating our features from our labels
X = final_result.drop("overall",axis=1) # Features
y = final_result["overall"] # Labels

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)


In [19]:
X_train.head()

,potential,value_eur,wage_eur,passing,dribbling,physic,movement_reactions,mentality_composure,foot_Left,foot_Right
158159,0.839736,-0.290013,-0.454842,0.095424,0.083989,0.114701,0.591402,0.902628,-0.553293,0.553293
112226,-0.598170,-0.352825,-0.356368,0.904323,0.083989,0.005855,0.373300,1.447336,-0.553293,0.553293
41296,1.638573,-0.135075,-0.081559,1.005435,-0.430632,0.114701,0.809504,-1.185422,-0.553293,0.553293
31461,-0.917705,-0.362875,-0.402170,-0.106801,-0.533556,-0.647227,-0.608157,0.085565,-0.553293,0.553293
57214,-0.438403,-0.264888,-0.310567,1.005435,0.083989,0.658936,0.700453,0.993412,1.807362,-1.807362


In [20]:
train_models = {
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, random_state=42),
    "GradientBoost": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

for name, estimator in train_models.items():
    cv_scores = cross_val_score(estimator, X_train, y_train, cv=5, scoring='r2')
    print(f"{name} Cross-Validation R2 Scores: {cv_scores}")
    print(f"{name} Mean Cross-Validation R2 Score: {np.mean(cv_scores)}")
    
    estimator.fit(X_train, y_train)
    r2_test_score = estimator.score(X_test, y_test)
    print(f"{name} Test R2 Score: {r2_test_score}\n")

RandomForest Cross-Validation R2 Scores: [0.96286966 0.96300406 0.96399316 0.96299226 0.96384255]
RandomForest Mean Cross-Validation R2 Score: 0.9633403381672947
RandomForest Test R2 Score: 0.9636009738531953

XGBoost Cross-Validation R2 Scores: [0.95791239 0.95796299 0.959369   0.95748925 0.95874703]
XGBoost Mean Cross-Validation R2 Score: 0.9582961320877075
XGBoost Test R2 Score: 0.957809567451477

GradientBoost Cross-Validation R2 Scores: [0.93993082 0.93937715 0.94148022 0.93798728 0.94176484]
GradientBoost Mean Cross-Validation R2 Score: 0.9401080637280128
GradientBoost Test R2 Score: 0.9391325078686747



# Training Models, Hypertuning and Evaluation

In [21]:
#Using Decision Tree Regression
X_train_set, X_test_set, y_train_set, y_test_set = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)

# Fit the model on the training data
dt_regressor.fit(X_train_set, y_train_set)

# Predict on the test data
y_test_predictions = dt_regressor.predict(X_test_set)

# Calculate R2 score and Mean Absolute Error
test_r2 = r2_score(y_test_set, y_test_predictions)
test_mae = mean_absolute_error(y_test_set, y_test_predictions)


In [22]:
print(f"Decision Tree Test R2 Score: {test_r2}")
print(f"Decision Tree Test MAE: {test_mae}")

Decision Tree Test R2 Score: 0.9261531874535888
Decision Tree Test MAE: 1.1210972553145404


In [23]:
#Using Linear Regression
X_train_data, X_test_data, y_train_data, y_test_data = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
linear_regressor = LinearRegression()

# Fit the model on the training data
linear_regressor.fit(X_train_data, y_train_data)

# Predict on the test data
y_test_predictions = linear_regressor.predict(X_test_data)

# Calculate R2 score and Mean Absolute Error
test_r2_score = r2_score(y_test_data, y_test_predictions)
test_mae_score = mean_absolute_error(y_test_data, y_test_predictions)



In [24]:
print(f"Linear Regression Test R2 Score: {test_r2_score}")
print(f"Linear Regression Test MAE: {test_mae_score}")

Linear Regression Test R2 Score: 0.8544813358027997
Linear Regression Test MAE: 2.0866007861820695


In [25]:
#Assessing the ensemble models using the evaluation metrics
for name, estimator in train_models.items():
    estimator.fit(X_train, y_train)

    y_train_predictions = estimator.predict(X_train)
    y_test_predictions = estimator.predict(X_test)

    train_mae_score = mean_absolute_error(y_train, y_train_predictions)
    test_mae_score = mean_absolute_error(y_test, y_test_predictions)
    test_r2_score = estimator.score(X_test, y_test)

    print(f"{name} Train MAE: {train_mae_score}")
    print(f"{name} Test MAE: {test_mae_score}")
    print(f"{name} Test R2 Score: {test_r2_score}\n")

RandomForest Train MAE: 0.3307216211483691
RandomForest Test MAE: 0.8933503292777746
RandomForest Test R2 Score: 0.9636009738531953

XGBoost Train MAE: 0.9982709041972779
XGBoost Test MAE: 1.0581460603301498
XGBoost Test R2 Score: 0.957809567451477

GradientBoost Train MAE: 1.296686450520165
GradientBoost Test MAE: 1.3076071977810326
GradientBoost Test R2 Score: 0.9391325078686747



# Hypertuning The Ensemble Model

In [26]:
X_train_set, X_test_set, y_train_set, y_test_set = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_estimator = train_models["XGBoost"]

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
}

random_search = RandomizedSearchCV(estimator=xgb_estimator,param_distributions=param_dist,n_iter=12,cv=5,scoring='r2',n_jobs=-1,random_state=42)

random_search.fit(X_train_set, y_train_set)

print(f"Best parameters for XGBoost: {random_search.best_params_}")
print(f"Best cross-validation R2 score for XGBoost: {random_search.best_score_}")

best_xgb_model = random_search.best_estimator_

best_xgb_model.fit(X_train_set, y_train_set)

y_train_predictions = best_xgb_model.predict(X_train_set)
y_test_predictions = best_xgb_model.predict(X_test_set)

train_r2_score = r2_score(y_train_set, y_train_predictions)
test_r2_score = r2_score(y_test_set, y_test_predictions)
train_mae_score = mean_absolute_error(y_train_set, y_train_predictions)
test_mae_score = mean_absolute_error(y_test_set, y_test_predictions)



Best parameters for XGBoost: {'learning_rate': 0.16743239807751675, 'max_depth': 9, 'n_estimators': 269}
Best cross-validation R2 score for XGBoost: 0.9645912408828735


In [27]:
print(f"XGBoost Train R2 Score: {train_r2_score}")
print(f"XGBoost Test R2 Score: {test_r2_score}")
print(f"XGBoost Train MAE: {train_mae_score}")
print(f"XGBoost Test MAE: {test_mae_score}")

XGBoost Train R2 Score: 0.9820632338523865
XGBoost Test R2 Score: 0.9647359848022461
XGBoost Train MAE: 0.6599097281771416
XGBoost Test MAE: 0.9361356736468912


# Test with new data set


In [28]:
players_22_df= pd.read_csv('players_22.csv',low_memory = False)

In [29]:
players_22_df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [30]:
#Test Data cleaning and Pre-processing
players_22_df = remove_weak_correlation(players_22_df, 'overall')

players_22_df = encode_preferred_foot(players_22_df)

players_22_df = remove_categorical_features(players_22_df)
players_22_df = drop_columns_with_high_nulls(players_22_df,null_threshold=0.3)
players_22_df = impute_with_median(players_22_df)
standardized_data, fitted_scaler = standardize_features(players_22_df, 'overall')
players_22_df = standardized_data

In [31]:
players_22_df

,overall,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,physic,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure,foot_Left,foot_Right
0,93,3.601780,9.889397,15.996581,9.589016,3.539198,3.542596,0.005012,2.217457,3.599846,2.139433,3.006228,3.131025,1.792891,-1.792891
1,92,3.437470,15.350116,13.424792,13.234253,2.275622,2.554865,1.844734,1.803392,3.489252,2.442649,1.980597,2.473077,-0.557758,0.557758
2,91,3.273160,5.547138,13.424792,5.385624,2.380920,2.774360,1.087202,1.458338,3.599846,2.745866,1.614300,3.048782,-0.557758,0.557758
3,91,3.273160,16.600160,13.424792,16.093937,3.012708,3.432848,-0.211426,1.872403,3.046874,1.684607,2.639931,2.884295,-0.557758,0.557758
4,91,3.273160,16.139617,17.539654,15.646035,3.749794,2.774360,1.411859,2.424490,3.268063,2.518454,2.932969,2.555321,-0.557758,0.557758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,-3.134932,-0.364913,-0.411430,-0.346563,-1.199211,-1.615558,-1.726492,-0.611986,-0.934526,-0.892735,-0.803260,-1.721343,-0.557758,0.557758
19235,47,-1.984762,-0.359649,-0.437148,-0.341119,-0.778020,-1.835054,-1.510054,-0.542976,-1.376903,-0.665323,-0.363704,-0.898908,-0.557758,0.557758
19236,47,-2.642002,-0.360965,-0.437148,-0.342360,-1.304509,-1.505810,-1.401835,-0.680997,-1.708687,-0.589519,-0.583482,-1.803587,-0.557758,0.557758
19237,47,-1.820452,-0.359649,-0.437148,-0.337949,-2.252191,-1.615558,-2.484025,-1.440116,-1.487498,-0.741127,-1.023038,-0.898908,-0.557758,0.557758


In [32]:
similar_columns = final_result.columns
players_info = players_22_df[similar_columns]

In [33]:
players_info.head()

,overall,potential,value_eur,wage_eur,passing,dribbling,physic,movement_reactions,mentality_composure,foot_Left,foot_Right
0,93,3.60178,9.889397,15.996581,3.539198,3.542596,0.005012,3.599846,3.131025,1.792891,-1.792891
1,92,3.43747,15.350116,13.424792,2.275622,2.554865,1.844734,3.489252,2.473077,-0.557758,0.557758
2,91,3.27316,5.547138,13.424792,2.380920,2.774360,1.087202,3.599846,3.048782,-0.557758,0.557758
3,91,3.27316,16.600160,13.424792,3.012708,3.432848,-0.211426,3.046874,2.884295,-0.557758,0.557758
4,91,3.27316,16.139617,17.539654,3.749794,2.774360,1.411859,3.268063,2.555321,-0.557758,0.557758


In [34]:
X_test.info

<bound method DataFrame.info of         potential  value_eur  wage_eur   passing  dribbling    physic  \
58706   -0.598170  -0.298388 -0.264765  0.095424   0.083989  0.114701   
92978   -0.278635  -0.248138  0.010044  0.600986   0.804458  0.985476   
66217   -0.118868  -0.347800 -0.310567 -0.915699   0.083989 -1.953389   
76576    0.839736  -0.185325 -0.264765  0.600986   0.804458  0.441242   
116129   0.040899  -0.218825 -0.457132  0.095424   0.083989  0.114701   
...           ...        ...       ...       ...        ...       ...   
152910   0.200667  -0.151825 -0.356368  0.196537   0.804458  0.223548   
30407   -0.118868  -0.354500 -0.402170 -2.533497  -2.592040  0.441242   
22270   -0.438403  -0.310950 -0.218964 -0.309025   0.392761 -1.191461   
127077   0.200667  -0.017825  0.651267  1.713222   1.216155  0.005855   
51057    1.798340   1.791175  3.170355  1.510997   1.422003  0.767783   

        movement_reactions  mentality_composure  foot_Left  foot_Right  
58706             

In [35]:
X_test, y_test = players_info, players_info['overall']
X_test = X_test.drop('overall', axis=1)

In [36]:
y_pred = best_xgb_model.predict(X_test)

In [37]:

m_s_e = mean_squared_error(y_test, y_pred)
r_m_s_e = np.sqrt(m_s_e)
m_a_e = mean_absolute_error(y_test, y_pred)
rsquared = r2_score(y_test, y_pred)

In [38]:
print(f"The Mean Squared Error is: {m_s_e}")
print(f"The Root Mean Squared Error is: {r_m_s_e}")
print(f"The Mean Absolute Error is: {m_a_e}")
print(f"The R-squared Score is: {rsquared}")

The Mean Squared Error is: 2.80660964901321
The Root Mean Squared Error is: 1.6752938992944522
The Mean Absolute Error is: 1.2511978845374736
The R-squared Score is: 0.9407076835632324


In [39]:
importance_feat = best_xgb_model.feature_importances_

# Create a DataFrame for feature importances
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance_feat
})

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the sorted DataFrame to show feature importances
print("Feature Importances:")
print(importance_df)

# Get the top features
t_features = importance_df.head(10)['Feature']
print("\nTop 10 Features:")
print(t_features)


Feature Importances:
               Feature  Importance
1            value_eur    0.766625
6   movement_reactions    0.105314
2             wage_eur    0.044406
5               physic    0.021900
0            potential    0.019395
7  mentality_composure    0.015694
4            dribbling    0.013195
3              passing    0.009852
8            foot_Left    0.003618
9           foot_Right    0.000000

Top 10 Features:
1              value_eur
6     movement_reactions
2               wage_eur
5                 physic
0              potential
7    mentality_composure
4              dribbling
3                passing
8              foot_Left
9             foot_Right
Name: Feature, dtype: object


In [40]:
# Downloading Scalar Used On Training and Test Data To Be Applied Onto User's Input
with open('fitscaler.pkl', 'wb') as file:
    pickle.dump(fitted_scaler, file)

In [42]:
with open('best_xgb_model.pkl', 'wb') as file:
    pickle.dump(best_xgb_model, file)